In [ ]:
!pip install strands-agents strands-agents-tools ddgs


In [ ]:
from strands import Agent
agent = Agent()
agent("안녕")

In [ ]:
from strands import tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    tools=[weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

agent("오늘 날씨 어때?")

In [ ]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

In [ ]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [ ]:
response = recipe_agent("닭고기와 브로콜리를 사용한 레시피를 제안해 주세요.")

# Other examples:
# response = recipe_agent("How do I cook quinoa?")
# response = recipe_agent("How can I substitute white wine in shrimp pasta?")
# response = recipe_agent("What are the health benefits of asparagus?")
print(response)

In [ ]:

from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

In [ ]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("Amazon Bedrock 가격모델은 어떻게 되나요? 간결히 설명해주세요.")